In [1]:
import matplobblib.AISD as ai 
ai.description('Структуры данных','Очередь через двусвязный список')


# Очередь через двусвязный список
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class Queue:
    def __init__(self):
        self.head = None
        self.tail = None

    def is_empty(self):
        return not bool(self.head)

    def enqueue(self, data):
        new_node = Node(data)
        if not self.head:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node

    def dequeue(self):
        data = self.head.data
        self.head = self.head.next
        if not self.head:
            self.tail = None
        return data

    def __len__(self):
        count = 0
        current = self.head
        while current:
            count += 1
            current = current.next
        return count

    def __str__(self):
        current = self.head
        queue_str = ""
        while current:
            queue_str += " → " + str(current.data)
   

In [52]:
import tkinter as tk
from heapq import heappop, heappush
import time

# Настройки
WIDTH, HEIGHT = 20, 20  # Размеры сетки
CELL_SIZE = 10  # Размер клетки в пикселях
WIDTH = WIDTH//2 *2+1
HEIGHT = HEIGHT//2 *2+1


# Цвета для визуализации
WALL_COLOR = "black"
EMPTY_COLOR = "white"
START_COLOR = "green"
END_COLOR = "red"
PATH_COLOR = "yellow"
VISITED_COLOR = "blue"

# Создаем лабиринт с пустыми клетками и стенами
maze = [[0 for _ in range(WIDTH)] for _ in range(HEIGHT)]
start = (1, 1)  # Начальная точка
end = (HEIGHT - 1, WIDTH - 1)  # Целевая точка

# Вспомогательные функции

def in_bounds(x, y):
    return 0 <= x < HEIGHT and 0 <= y < WIDTH

def neighbors(x, y):
    """ Возвращает соседние клетки (вверх, вниз, влево, вправо) """
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < HEIGHT and 0 <= ny < WIDTH:
            yield nx, ny
            
def generate_maze(start_x, start_y):
    # Начинаем с произвольной точки
    maze[start_y+10][start_x] = 1

def heuristic(a, b):
    """ Эвристика: манхэттенское расстояние """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(start, goal):
    """ Реализация алгоритма A* """
    open_list = []
    heappush(open_list, (0, start))
    came_from = {}
    g_score = {start: 0}
    
    while open_list:
        _, current = heappop(open_list)
        
        if current == goal:
            return reconstruct_path(came_from, current)
        
        x, y = current
        canvas.itemconfig(rects[y][x], fill=VISITED_COLOR)
        root.update()
        time.sleep(0.1)
        
        for neighbor in neighbors(x, y):
            nx, ny = neighbor
            if maze[nx][ny] == 1:  # Если стена, пропускаем
                continue
            
            tentative_g_score = g_score[current] + 1
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor, goal)
                heappush(open_list, (f_score, neighbor))
                came_from[neighbor] = current
    
    return []

def reconstruct_path(came_from, current):
    """ Восстанавливаем путь по пришедшим координатам """
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path

def draw_path(path):
    """ Отображаем найденный путь """
    for (x, y) in path:
        canvas.itemconfig(rects[y][x], fill=PATH_COLOR)
        root.update()
        time.sleep(0.01)

# Визуализация с помощью Tkinter
root = tk.Tk()
root.title("A* Pathfinding Visualization")



canvas = tk.Canvas(root, width=WIDTH * CELL_SIZE, height=HEIGHT * CELL_SIZE)
canvas.pack()

rects = [[None for _ in range(WIDTH)] for _ in range(HEIGHT)]
generate_maze(0,0)
for i in range(HEIGHT):
    for j in range(WIDTH):
        color = EMPTY_COLOR if maze[i][j] == 0 else WALL_COLOR
        if color == WALL_COLOR:
            print(f'Wall, {i,j}')
        
        rect = canvas.create_rectangle(j * CELL_SIZE, i  * CELL_SIZE, (j + 1) * CELL_SIZE, (i + 1) * CELL_SIZE, fill=color, outline="gray")
        rects[i][j] = rect

# Обозначаем начальную и конечную точки
sx, sy = start
ex, ey = end
canvas.itemconfig(rects[sy][sx], fill=START_COLOR)
canvas.itemconfig(rects[ey][ex], fill=END_COLOR)

# Запуск алгоритма A*
path = a_star(start, end)
if path:
    draw_path(path)
else:
    print("Путь не найден")

root.mainloop()


Wall, (10, 0)


TclError: invalid command name ".!canvas"

In [4]:
import tkinter as tk
from  tqdm.notebook import tqdm
from random import randint
from scipy.stats import uniform
from heapq import heappop, heappush
import time

# Настройки
WIDTH, HEIGHT = 20, 20  # Размеры сетки
CELL_SIZE = 10  # Размер клетки в пикселях
WIDTH = WIDTH//2 *2+1
HEIGHT = HEIGHT//2 *2+1


# Цвета для визуализации
WALL_COLOR = "black"
EMPTY_COLOR = "white"
START_COLOR = "green"
END_COLOR = "red"
PATH_COLOR = "yellow"
VISITED_COLOR = "blue"

# Создаем лабиринт с пустыми клетками и стенами
maze = [[0 for _ in range(WIDTH)] for _ in range(HEIGHT)]
start = (1, 1)  # Начальная точка
end = (HEIGHT - 1, WIDTH - 1)  # Целевая точка

# Вспомогательные функции
class Cell():
    def __init__(self, x,y,if_not_wall=0,been_here=0):
        self.x=x
        self.y=y
        self.if_not_wall = if_not_wall ### 1 - проход, 0 - стена
        self.been_here=been_here       ### 1 - посещена, 0 - не посещена
        
def in_bounds(x, y):
    return 0 <= x < HEIGHT and 0 <= y < WIDTH

def neighbors(x, y,distance=1):
    """ Возвращает соседние клетки (вверх, вниз, влево, вправо) """
    
    for dx, dy in [(-distance, 0), (distance, 0), (0, -distance), (0, distance)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < HEIGHT and 0 <= ny < WIDTH:
            if maze[ny][nx].if_not_wall==1 and maze[ny][nx].been_here==0:
                yield maze[ny][nx]

def removeWall(cell_1,cell_2):
    delta_x=cell_1.x-cell_2.x
    delta_y=cell_1.y-cell_2.y
    add_x = (delta_x / abs(delta_x)) if delta_x!=0 else 0 
    add_y = (delta_y / abs(delta_y)) if delta_y!=0 else 0
    maze[cell_1.x+add_x][cell_1.y+add_y].been_here=1
    maze[cell_1.x+add_x][cell_1.y+add_y].if_not_wall=1
    
def generate_maze(start_x, start_y):
    # Начинаем с произвольной точки
    for i in tqdm(range(HEIGHT),'Сделаем изначальную расстановку'):
        for j in range(WIDTH):
            if i%2!=0 and j%2!=0:
                maze[i][j]=Cell(j,i,1)
            else:
                maze[i][j]=Cell(j,i,0)
    start_cell=(start_x, start_y)
    c_cell=start_cell
    for i in tqdm(range(len(maze)*len(maze[0])),'Прорубим-ка стен'):
        neighbours= list(neighbors(*c_cell,))
        if len(neighbours)!=0:
            print(uniform(0,len(neighbours)-1).rvs(1))
            
            
    
    

# Визуализация с помощью Tkinter
root = tk.Tk()
root.title("A* Pathfinding Visualization")



canvas = tk.Canvas(root, width=WIDTH * CELL_SIZE, height=HEIGHT * CELL_SIZE)
canvas.pack()

rects = [[None for _ in range(WIDTH)] for _ in range(HEIGHT)]
generate_maze(0,0)
for i in range(HEIGHT):
    for j in range(WIDTH):
        color = EMPTY_COLOR if maze[i][j].if_not_wall==1 else WALL_COLOR
        if color == WALL_COLOR:
            pass
            #print(f'Wall, {i,j}')
        
        rect = canvas.create_rectangle(j * CELL_SIZE, i  * CELL_SIZE, (j + 1) * CELL_SIZE, (i + 1) * CELL_SIZE, fill=color, outline="gray")
        rects[i][j] = rect

# Обозначаем начальную и конечную точки
sx, sy = start
ex, ey = end
canvas.itemconfig(rects[sy][sx], fill=START_COLOR)
canvas.itemconfig(rects[ey][ex], fill=END_COLOR)


root.mainloop()

Сделаем изначальную расстановку:   0%|          | 0/21 [00:00<?, ?it/s]

Прорубим-ка стен:   0%|          | 0/441 [00:00<?, ?it/s]